## <font color="#004D7F">Ejercicio Wisconsin </font>

In [ ]:
#Import libraries
import numpy as np
import pandas as pd
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

from sklearn import datasets
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
# from sklearn.cross_validation import KFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from scipy.interpolate import spline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_ridge import KernelRidge

In [ ]:
# Wisconsin
import sklearn.datasets
wisconsin = sklearn.datasets.load_breast_cancer()
wisconsin.keys()
wisconsin_data = wisconsin['data']
wisconsin_target = wisconsin['target']

# creamos un objeto con los parámetros por defecto
#lr = LogisticRegression(solver='lbfgs', tol = 0.1)
lr = LogisticRegression()

# entrenamos con los datos de entrada y la salida
lr.fit(wisconsin_data,wisconsin_target)

# obtenemos una predicción para los datos de pima
wisconsin_prediction = lr.predict(wisconsin_data)

# comparamos estos datos con el error cuadrático medio
from sklearn.metrics import mean_squared_error
print('Error cuadrático medio:')
print(mean_squared_error(wisconsin_target, wisconsin_prediction))

# y obtenemos directamente el score
print('Coeficiente R2 de la función score:')
print(lr.score(wisconsin_data,wisconsin_target))

## <font color="#004D7F">Ejercicio MNIST </font>

In [ ]:
from sklearn.datasets import fetch_openml

# esta base de datos se puede descargar directamente de
# https://www.openml.org/d/554
# y le podemos indicar la ubicación en la que queremos que se almacene con data_home
mnist = fetch_openml('mnist_784', version=1, data_home=".")

# la base de datos descargada consiste en un diccionario con diferentes parámetros, de los
# cuales sólo nos quedamos con data y target.

# data consiste en una matriz donde cada fila es una imagen y las columnas representan los
# pixels que la componen
mnist_data = mnist['data']
print("Número de imágenes: {0}\tPixels por imagen: {1}".format(mnist_data.shape[0],mnist_data.shape[1]))

# target contiene las etiquetas de cada una de las imágenes, donde el valor de la clase 
# corresponde con el dígito que se muestra en la imagen
mnist_target = mnist['target']
print("Número de etiquetas: {0}".format(mnist_target.shape[0]))

# creamos un objeto con los parámetros por defecto
#lr = LogisticRegression(solver='lbfgs', tol = 0.1, max_iter=5)
lr = LogisticRegression(tol = 0.1)
#lr = LogisticRegression()

# entrenamos con los datos de entrada y la salida
lr.fit(mnist_data,mnist_target)

# obtenemos una predicción para los datos de pima
mnist_prediction = lr.predict(mnist_data)

# comparamos estos datos con el error cuadrático medio
from sklearn.metrics import mean_squared_error
print('Error cuadrático medio:')
print(mean_squared_error(mnist_target, mnist_prediction))

# y obtenemos directamente el score
print('Coeficiente R2 de la función score:')
print(lr.score(mnist_data,mnist_target))

## <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Matriz de confusion - Teoria</font> 

In [ ]:
import sklearn.datasets
wisconsin = sklearn.datasets.load_breast_cancer()
wisconsin.keys()
wisconsin_data = wisconsin['data']
wisconsin_target = wisconsin['target']
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
X = wisconsin_data
Y = wisconsin_target
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

lr = LogisticRegression()

lr.fit(X_train, Y_train)
predicciones = lr.predict(X_test)
print('Tasa de acierto o accuracy clasificador en el entorno de test: {:.2f}'.format(lr.score(X_test, Y_test)))

# Matriz de confusion
mc= confusion_matrix(y_true= Y_test, y_pred=predicciones)
clases = np.unique(Y)
print('Clase real\n  {:}   {:}'.format(clases[0],clases[1]))
print(mc)
print('\nTP={:d} TN={:d}'.format(mc[0,0], mc[1,1]))
print('FP={:d}  FN={:d}'.format(mc[1,0], mc[0,1]))


## <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio:  Análisis de datos en <i>Marketing</i></font> 

El objetivo final consiste en elaborar un modelo de detección del abandono. Estos pasos pueden servir de orientación en el desarrollo:

- En este caso, se parte de un solo conjunto de datos, que se debe dividir en una partición de entrenamiento y otra de test, que se utilizará para validar.

- Como paso previo, es puede ser interesante explorar la distribución de la clase. Para ello, puede hacerse, por ejemplo, un gráfico de barras.

- Por último, habría que reportar el rendimiento (en acierto y coste) sobre el conjunto de test.

Para la realización del ejercicio, se proporciona un conjunto de datos, denominado, `datos/churn/customerChurnAnalysis.csv`, relativo al abandono en una operadora de telecomunicaciones. 

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
df = pd.read_csv('../1-ScikitLearn/datos/churn/customerChurnAnalysis.csv')
df.shape

In [ ]:
df.head(3)
df.columns

In [ ]:
# Check the correlation
df.corr()
#Visualize correlation between attributes by using heatmap
sns.heatmap(df.corr())

In [ ]:
# Feature contains high correlation. We need to remove them first before applying regression techniques.
# Create correlation matrix
abs_corr_matrix = df.corr().abs()

In [ ]:
# Select upper triangle of matrix
up_tri = abs_corr_matrix.where(np.triu(np.ones(abs_corr_matrix.shape), k=1).astype(np.bool))

# Find all the features which is having correlation > 0.75 with other features.
correlated_features = [column for column in up_tri.columns if any(up_tri[column] > 0.75)]

#Print correlated_features
print(correlated_features)

In [ ]:
df = df.drop(correlated_features, axis=1)

In [ ]:
# Pensar en hacer la correlación de columnas? 
#df.drop(['state','international plan','voice mail plan'], axis=1, inplace=True)

In [ ]:
from sklearn import preprocessing
## Aqui una función para que cambie todos los object
# Funcion no válida para boolean porque genera una matriz erronea
## Este es el error que devuelve: LinAlgError: Singular matriz

def encoder(df):
    '''Manejo de campos categoricos para poder ser utilizados en scikit Learn'''
    for col in df.columns:
        if df.dtypes[col] == 'object':
            label_encoder = preprocessing.LabelEncoder()
            df[col] = label_encoder.fit_transform(df[col])
    return df

In [ ]:
encoder(df)

In [ ]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
df['churn'] = label_encoder.fit_transform(df['churn'])

In [ ]:
# Tras eliminar variables muy correlacionadas
# Podemos hacer las particiones en arrays Numpy 
# Y = df.iloc[:,16].values
# X = df.iloc[:,0:16].values

In [ ]:
from sklearn.model_selection import train_test_split
# Particiones train y test total day charge', 'total eve charge', 'total night charge', 'total intl charge
y = df["churn"]
X = df[['state','account length','area code','phone number','international plan','number vmail messages','total day minutes',
           'total day calls','total eve minutes','total eve calls','international plan','total night minutes',
           'total night calls','total intl minutes','total intl calls','voice mail plan','customer service calls']]
   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print("Partición para train: ", X_train.shape, y_train.shape)
print("Partición para test: ", X_test.shape , y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy del clasificador en el entorno de test: {:.2f}'.format(model.score(X_test, y_test)))

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
import scikitplot as skplt
skplt.estimators.plot_learning_curve(logistic_regression, X_train, y_train)

In [ ]:
# Fit el modelo con la norma L1

logistic_regression_l1 = LogisticRegression(penalty='l1',class_weight='balanced')
logistic_regression_l1.fit(X_train, y_train)

#Check on test data:

y_pred = logistic_regression_l1.predict(X_test)
print(accuracy_score(y_test, y_pred))

#ROC Curve:

y_pred_prob_l1 = logistic_regression_l1.predict_proba(X_test)
class_1_prob = list()
for i in y_pred_prob_l1:
    class_1_prob.append(i[1])
print(roc_auc_score(y_test, class_1_prob))

model_result['Logistic Regression (L1)'] = roc_auc_score(y_test, class_1_prob)

skplt.metrics.plot_roc_curve(y_test, y_pred_prob_l1, curves =['each_class'])


In [ ]:
y_pred_prob = model.predict_proba(X_test)

#ROC curve:

class_1_prob = list()
for i in y_pred_prob:
    class_1_prob.append(i[1])
print(roc_auc_score(y_test, class_1_prob))

model_result['Logistic Regression (L2)'] = roc_auc_score(y_test, class_1_prob)

skplt.metrics.plot_roc_curve(y_test, y_pred_prob, curves=['each_class'])

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)

Gráfica matriz de confusión con seaborn

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

sns.heatmap(confusion_matrix, annot=True,  fmt='');
title = 'Matriz de Confusión sobre Regresión Logística con fit regularizado a 0.5'
plt.title(title);

Gráfica matriz de confusión con scikit plot

In [ ]:
skplt.metrics.plot_confusion_matrix(y_test, y_pred, normalize=True)

###  Churn sin columas categoricas

In [ ]:
df.to_csv('../1-ScikitLearn/datos/churn/customerChurnAnalysisModified.csv')
#df.to_csv(.\datos)

In [ ]:
from sklearn.model_selection import train_test_split

y = df["churn"]
X = df[['account length','area code','number vmail messages','total day minutes',
           'total day calls','state','total eve minutes','total eve calls','international plan','total night minutes',
           'total night calls','total intl minutes','total intl calls','voice mail plan','customer service calls']]
   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

print("Partición para train: ", X_train.shape, y_train.shape)
print("Partición para test: ", X_test.shape , y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import metrics

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print('Accuracy del clasificador en el entorno de test: {:.2f}'.format(model.score(X_test, y_test)))

In [ ]:
confusion_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

sns.heatmap(confusion_matrix, annot=True,  fmt='');
title = 'Matriz de Confusión sobre Regresión Logística con fit regularizado a 0.5'
plt.title(title);

## <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Titanic</font> 

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score
from sklearn import metrics

import scikitplot as skplt

#Hide all the warnings in jupyter notebook
import warnings
warnings.filterwarnings('ignore')

# we will build different model and compare them later. Let's store the result (AUC Score) in a dictionary.

#Store result from different models
model_result = {}

In [ ]:
#Load data in pandas dataframe
titanic_data = pd.read_csv('../1-ScikitLearn/Titanic.csv', sep='\t')

In [ ]:
titanic_data.head()

In [ ]:
#Keep selected columns
titanic_data = titanic_data[['Survived','Pclass','Sex','Age','SibSp','Parch','Embarked','Fare']]
titanic_data.shape

In [ ]:
titanic_data['Survived'].value_counts()
sns.countplot(x='Survived',data=titanic_data)

In [ ]:
titanic_data.isnull().any()

In [ ]:
titanic_data.isnull().sum()

In [ ]:
titanic_data['Age'] .fillna ((titanic_data['Age'] .mean()), inplace=True)
titanic_data.dropna(inplace=True)

In [ ]:
# Now check the data type of each feature:
titanic_data.dtypes

In [ ]:
titanic_data_X = titanic_data[['Pclass','Sex','Age','SibSp','Parch', 'Embarked', 'Fare']]
titanic_data_Y = titanic_data[['Survived']]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(titanic_data_X, titanic_data_Y, test_size=0.20)


In [ ]:
sns.countplot(x='Pclass',data=X_train)
titanic_data_Y = titanic_data[['Survived']]

In [ ]:
#Check the distribution of age data.
sns.distplot(X_train['Age'])

In [ ]:
sns.distplot(X_train['Fare'])


In [ ]:
# We will perform Z-Score normalization on both these features. Age & Fare

age_scaler = StandardScaler()
age_scaler.fit(pd.DataFrame(X_train['Age']))

In [ ]:
X_train[['Age']] = age_scaler.transform(X_train[['Age']])

In [ ]:
fare_scaler = StandardScaler()
fare_scaler.fit(pd.DataFrame(X_train['Fare']))
X_train[['Fare']] = fare_scaler.transform(X_train[['Fare']])

In [ ]:
# Change sex feature to 0,1 value.

X_train['Sex'] = X_train['Sex'].map({'female': 0, 'male':1})


In [ ]:
# Embarked has 3 categories. We can create three different variable which represents each category.
embarked_encoder = preprocessing.LabelEncoder()
embarked_encoder.fit(pd.DataFrame(X_train['Embarked']))
X_train[['Embarked']] = embarked_encoder.transform(X_train[['Embarked']])


In [ ]:
# correlación entre variables
sns.heatmap(X_train.corr())

In [ ]:
#Fare and Pclass has high correlation.
# In logistic regression features should not be correlated. So, we can remove one variable.
del X_train['Pclass']


In [ ]:
# Modelo
X_train_original = X_train
X_train = X_train.values
Y_train = Y_train.values
logistic_regression = LogisticRegression()
logistic_regression.fit(X_train,Y_train)
coefficients = pd.concat([pd.DataFrame(X_train_original.columns),pd.DataFrame(np.transpose(logistic_regression.coef_))],axis = 1)
coefficients.columns = ['Feature','coefficient']
coefficients = coefficients.append({'Feature':'Intercept','coefficient':logistic_regression.intercept_[0]}, ignore_index=True)
coefficients


In [ ]:
# Our model is created. We can test it on our test data. But first we need to transform the test data same as we did with training data.

def transform_test_data(test_data,age_scaler,fare_scaler,embarked_encoder):
    test_data['Sex'] = test_data['Sex'].map({'female': 0,'male': 1})
    test_data[['Age']] = age_scaler.transform(test_data [['Age']])
    test_data[['Fare']] = fare_scaler.transform(test_data [['Fare']])
    test_data[['Embarked']] = embarked_encoder.transform(test_data[['Embarked']])
    del test_data['Pclass']
    return test_data


In [ ]:
X_test = transform_test_data(X_test,age_scaler,fare_scaler,embarked_encoder)

X_test = X_test.values
Y_test = Y_test.values


In [ ]:
Y_pred = logistic_regression.predict(X_test)

In [ ]:
confusion_matrix(Y_test,Y_pred)


In [ ]:
print(classification_report(Y_test, Y_pred))

In [ ]:
# Accuracy
print(accuracy_score(Y_test,Y_pred))

In [ ]:
# will plot different classification result by using Scikit-plot library.
# pip install scikit-plot

In [ ]:
import scikitplot as skplt
skplt.estimators.plot_learning_curve(logistic_regression, X_train,Y_train)

In [ ]:
Y_pred_prob = logistic_regression.predict_proba(X_test)

#ROC curve:

class_1_prob = list()
for i in Y_pred_prob:
    class_1_prob.append(i[1])
print(roc_auc_score(Y_test,class_1_prob))

model_result['Logistic Regression (L2)'] = roc_auc_score(Y_test,class_1_prob)

skplt.metrics.plot_roc_curve(Y_test, Y_pred_prob, curves=['each_class'])


In [ ]:
skplt.metrics.plot_confusion_matrix(Y_test, Y_pred, normalize=True)

In [ ]:
# Fit the model with L1 norm:

logistic_regression_l1 = LogisticRegression(penalty='l1',class_weight='balanced')
logistic_regression_l1.fit(X_train,Y_train)

#Check on test data:

Y_pred = logistic_regression_l1.predict(X_test)
print(accuracy_score(Y_test,Y_pred))

#ROC Curve:

Y_pred_prob = logistic_regression_l1.predict_proba(X_test)
class_1_prob = list()
for i in Y_pred_prob:
    class_1_prob.append(i[1])
print(roc_auc_score(Y_test,class_1_prob))

model_result['Logistic Regression (L1)'] = roc_auc_score(Y_test,class_1_prob)

skplt.metrics.plot_roc_curve(Y_test, Y_pred_prob,curves =['each_class'])
